In [918]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [919]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [920]:
train = pd.read_csv('../data/prep/train3_train.csv')
test = pd.read_csv('../data/prep/test3_valid.csv')
submission = pd.read_csv('../data/prep/submission_valid.csv')

In [921]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [922]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [923]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [924]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [925]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [926]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [912]:
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2892852	test: 183381.3000474	best: 183381.3000474 (0)	total: 1.1ms	remaining: 3.3s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 183381.2629
bestIteration = 133

Shrink model to first 134 iterations.
1 FOLD NMAE = 0.08523984975261902

2 FOLD Training.....
0:	learn: 0.2905352	test: 188228.8769451	best: 188228.8769451 (0)	total: 1.11ms	remaining: 3.32s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 188228.7988
bestIteration = 207

Shrink model to first 208 iterations.
2 FOLD NMAE = 0.10384232671632576

3 FOLD Training.....
0:	learn: 0.2892624	test: 176123.6074464	best: 176123.6074464 (0)	total: 1.73ms	remaining: 5.18s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 176123.6074
bestIteration = 0

Shrink model to first 1 iterations.
3 FOLD NMAE = 0.29453394284470136

4 FOLD Training.....
0:	learn: 0.2906645	test: 192980.9502626	best: 192980.9502626 (0)	total: 1.63ms	remaining: 4.88s
1000:	learn: 0.

In [927]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.07898627274687313

2 FOLD Training.....
2 FOLD NMAE = 0.09947348602294435

3 FOLD Training.....
3 FOLD NMAE = 0.10633357406336943

4 FOLD Training.....
4 FOLD NMAE = 0.08698153853374317

5 FOLD Training.....
5 FOLD NMAE = 0.10210106830469523

6 FOLD Training.....
6 FOLD NMAE = 0.10326260790512203

7 FOLD Training.....
7 FOLD NMAE = 0.1087532068413808

8 FOLD Training.....
8 FOLD NMAE = 0.0935836114863561

9 FOLD Training.....
9 FOLD NMAE = 0.107240454537661

10 FOLD Training.....
10 FOLD NMAE = 0.08872926978028181

10FOLD Mean of NMAE = 0.09754450902224271 & std = 0.009505027098910072


In [928]:
submission['target'] = ngb_pred

In [929]:
submission.to_csv("../out/ngb/ngb1_valid.csv", header = True, index = False)

In [930]:
submission['target'] = cb_pred

In [931]:
submission.to_csv("../out/cb/cb1_valid.csv", header = True, index = False)